## Предыстория

Для сериализации объектов в языке `python` используется модуль [`pickle`](https://docs.python.org/3/library/pickle.html). Так с помощью этого модуля можно сохранять, например, модели машинного обучения или просто любые другие данные.

В директории `sphere_data` лежат несколько файлов с `python`-объектами, сериализованными с помощью `pickle`. Каждый файл имеет следующий формат:
* N – количество объектов в файле;
* последовательность из N словарей (`record`).

Пример формирования файла `sphere_data/part_*.pkl`:

```python
data = [...]

with open('sphere_data/part_00000.pkl', 'wb') as f_out:
    pickle.dump(len(data), f_out)
    for record in data:
        pickle.dump(data, f_out)
```

Каждый словарь имеет вид:
```python
record = {
    'app_id': int,
    'sequence': list,
    'sequence_cat': list,
    'sequence_len': int,
    'product': int,
    'flag': int,
}
```

* `app_id` – некоторый идентификатор, имеющий тип `int`.
* `sequence` – двумерный массив, внутренний массив имеет длину `sequence_len` и хранит элементы типа `float`; массив представляет собой список вложенных списков (`list`).
* `sequence_cat` – двумерный массив, аналогичный `sequence`, хранит элементы типа `int`;
* `sequence_len` – длина вложенных списков: `len(sequence[0]) == len(sequence_cat[0]) == sequence_len`.
* `product` и `flag` – некоторые категориальные переменные типа `int`.

Чтобы считать объект из файла используйте `pickle.load`.

## Задание

В данном задании требуется написать генератор
```python
def app_records(path='sphere_data', shuffle=False, random_state=None, chunk_size=100, max_sequence_len=750):
    pass
```

Генератор `app_records` может представлять собой генератор функцию или быть классом, реализующим механизм итерации. Вы можете выбрать наиболее удобный вариант для вас.

### Пункт 1. Основная логика (4 балла)

Генератор должен зачитывать данные из файлов, лежащих в директории `path`. Гарантируется, что все файлы в директории будут иметь формат, описанный выше и будут называться аналогично: `part_[0-9]{5}.pkl`. 

В случае, если `shuffle=False` требуется осортировать все файлы в директории `path` в лексикографическом порядке и отдавать по одной записи из каждого файла. Если очередной файл закончился, требуется открыть следующий файл, и начать отдавать данные из него.

Таким образом, с помощью, например, вот такого кода можно получить все записи из файлов в исходном порядке:
```python
reader = app_records(path='sphere_data', shuffle=False)
for record in reader:
    # do something with record
```

### Пункт 2. Перемешивание (2 балла)

Генератор должен уметь перемешивать порядок записей. В общем случае содержимое всех файлов может быть достаточно большим, что не поместится в оперативную память компьютера. В связи с этим предлагается реализовать следующий алгоритм псевдо-перемешивания:
* перемешиваем файлы (порядок, в котором будем их обрабатывать);
* считываем из перемешанных файлов `chunk_size` записей;
* перемешиваем считанные записи.

Далее генератор по-прежнему отдает записи по одной.

Для воспроизводимости результата перемешивания нужно использовать аргумент `random_state` (начальный seed). Для перемешивания лучше использовать модуль [`random`](https://docs.python.org/3/library/random.html), но допускается использование модуля [`numpy.random`](https://numpy.org/doc/stable/reference/random/index.html).

### Пункт 3.  Паддинг (4 балла)

Генератор должен делать паддинг для массивов `sequence` и `sequence_cat` (дополнение нулями), если длина вложенного списка меньше, чем `max_sequence_len`: `len(sequence[0]) < max_sequence_len`.

Пример:

```python
sequence = [
    [0, 3, 3, 1, 3, 1],
    [4, 3, 4, 3, 2, 3],
    [2, 0, 0, 3, 3, 2],
]

assert np.asarray(sequence).shape == (3, 6)
assert sequence_len == 6
assert max_sequence_len == 10
```

Тогда паддинг для массива `sequence` должен выглядеть как:

```python
sequence = [
    [0, 3, 3, 1, 3, 1, 0, 0, 0, 0],
    [4, 3, 4, 3, 2, 3, 0, 0, 0, 0],
    [2, 0, 0, 3, 3, 2, 0, 0, 0, 0],
]

assert np.assaray(sequence).shape == (3, 10)
```

Если `len(sequence[0]) > max_sequence_len`, то нужно обрезать массив до `max_sequence_len`.

Пример:

```python
sequence = [
    [0, 3, 3, 1, 3, 1],
    [4, 3, 4, 3, 2, 3],
    [2, 0, 0, 3, 3, 2],
]

assert np.asarray(sequence).shape == (3, 6)
assert sequence_len == 6
assert max_sequence_len == 4
```

Тогда обрезанный для массив `sequence` должен выглядеть как:

```python
sequence = [
    [0, 3, 3, 1],
    [4, 3, 4, 3],
    [2, 0, 0, 3],
]

assert np.assaray(sequence).shape == (3, 4)
```

В этом пункте обязательно использовать библиотеку `numpy` для работы с формой массива. Результат применения паддинга или обрезания массива должен быть `np.ndarray`.